## Selección de características para mejorar modelos predictivos

En este cuadernillo se realiza el tratamiento de datos del csv TITANIC, así como la implementación y comparación de varios algoritmos de búsqueda y entrenamiento.

### Tratamiento de datos(Hay que CAMBIAR)

Se aplicaron los siguientes procesos de preprocesamiento de datos:

    •Normalización de variables predictoras: Se normalizaron las variables Age y Fare utilizando el escalador MinMaxScaler para asegurar que todas las características estén en la misma escala.

    •Codificación numérica de atributos discretos: Los atributos Sex, Embarked, Alone y Deck, que originalmente se presentaban como cadenas de texto, fueron codificados numéricamente utilizando las técnicas de OrdinalEncoder o LabelEncoder, según corresponda.

### Primer experimento para evaluar la capacidad predictiva del conjunto de variables completo

    Para evaluar la calidad de las soluciones que se obtienen con los algoritmos de búsqueda de variables, se realizó un primer experimento en el que se entrenó un modelo de clasificación utilizando todas las variables predictoras disponibles. Se utilizó un árbol de decisión de clasificación.

### Algoritmos de búsqueda implementados

    •Búsqueda secuencial hacia atrás (backward_sequential_search): Este algoritmo busca encontrar el mejor subconjunto de variables predictoras eliminando iterativamente la variable que más afecta el rendimiento del modelo.

    •Búsqueda secuencial hacia atrás mixta (backward_sequential_search_mixto): Similar al anterior, pero también considera añadir variables si se mejora el rendimiento del modelo.

### Algoritmos de entrenamiento

    •Árboles de decisión de clasificación (DecisionTreeClassifier): Un algoritmo de aprendizaje supervisado utilizado para clasificación.




## Tratamiento de datos

In [1]:
import numpy as np
np.random.seed(357823)

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler


model = RandomForestClassifier()

breast_cancer = pd.read_csv('../data/BreastCancer.csv')
breast_cancer.head()

atributos_continuos = ['mean radius' ,'mean texture' ,'mean perimeter' ,'mean area' ,'mean smoothness' ,'mean compactness' ,'mean concavity' ,
                       'mean concave points' ,'mean symmetry' ,'mean fractal dimension' ,'radius error' ,'texture error' ,'perimeter error' ,
                       'area error' ,'smoothness error' ,'compactness error' ,'concavity error' ,'concave points error','symmetry error','fractal dimension error',
                       'worst radius' ,'worst texture' ,'worst perimeter','worst area' ,'worst smoothness','worst compactness' ,'worst concavity' ,'worst concave points','worst symmetry','worst fractal dimension']
atributos = breast_cancer.loc[:, atributos_continuos]

objetivo = breast_cancer['diagnosis']
objetivo.head()  # objetivo es una Series unidimensional

normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)

# Como nos interesa conservar los atributos originales, realizamos la
# normalización sobre una copia del DataFrame de atributos
atributos_normalizados = atributos.copy()
atributos_normalizados[:] = normalizador.fit_transform(atributos_normalizados)
atributos_normalizados.head()

breast_cancer = atributos_normalizados.copy()
breast_cancer['Diagnosis'] = objetivo
breast_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Diagnosis
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864,0
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878,0
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433,0
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711,0
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595,0


## Evaluación del modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate


(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

clasificador_CART = DecisionTreeClassifier(random_state=42)
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'min_samples_split': range(5, 20, 5)
}
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)
print("Mejores parámetros:", búsqueda_en_rejilla.best_params_)
print("Mejor score:", búsqueda_en_rejilla.best_score_)


NameError: name 'atributos_entrenamiento' is not defined

In [4]:
resultados_validación_cruzada = cross_validate(clasificador_CART,
                                               atributos_entrenamiento,
                                               objetivo_entrenamiento,
                                               scoring='balanced_accuracy',
                                               cv=10, 
                                               n_jobs=-1,)
#resultados_validación_cruzada
resultados_validación_cruzada['test_score'].mean()

0.9166075050709941

In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

# Supongamos que atributos es un DataFrame con las características y objetivo es una Serie con la variable objetivo
# Ajustar el modelo
clasificador_CART = DecisionTreeClassifier(max_depth=4)
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Calcular la precisión del modelo
score = clasificador_CART.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_CART.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

# Imprimir los resultados
print(f'Variabes predictoras: \n{atributos_entrenamiento.columns}')
print()
print(f'Precisión: {score}')
print()
print(f'Valores: {values}')
print()
print(f'Matriz de confusión: \n{m_confusion}')


Variabes predictoras: 
Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')

Precisión: 1.0

Valores: diagnosis
1    357
0    212
Name: count, dtype: int64

Matriz de confusión: 
[[212   0]
 [  0 357]]


### KNN

In [6]:
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer([('normalizador',
                                   MinMaxScaler(feature_range=(0, 1)),
                                   atributos_continuos)])

Variabes predictoras:      mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0          17.99         10.38          122.80     1001.0          0.11840   
1          20.57         17.77          132.90     1326.0          0.08474   
2          19.69         21.25          130.00     1203.0          0.10960   
3          11.42         20.38           77.58      386.1          0.14250   
4          20.29         14.34          135.10     1297.0          0.10030   
..           ...           ...             ...        ...              ...   
564        21.56         22.39          142.00     1479.0          0.11100   
565        20.13         28.25          131.20     1261.0          0.09780   
566        16.60         28.08          108.30      858.1          0.08455   
567        20.60         29.33          140.10     1265.0          0.11780   
568         7.76         24.54           47.92      181.0          0.05263   

     mean compactness  mean concavity  me

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

tubería_kNN = Pipeline([('preprocesador', normalizador),
                        ('kNN', KNeighborsClassifier())])
rejilla_de_parámetros = {
    # Número de vecinos impar (tarea de clasificación binaria)
    'kNN__n_neighbors': range(1, 10, 2),
    # Considerar las distancias Manhattan y euclídea
    'kNN__metric': ['manhattan', 'euclidean']
}


búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

print("Mejores parámetros:", búsqueda_en_rejilla.best_params_)
print("Mejor score:", búsqueda_en_rejilla.best_score_)

In [ ]:
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 5 ejemplos más cercanos
    n_neighbors=5,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)


# Calcular la precisión del modelo
score = clasificador_kNN.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_kNN.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

print(f'Variabes predictoras: {atributos_entrenamiento.columns}')
print(f'Precisión: {score}')
print(f'Valores: {values}')
print(f'Matriz de confusión: \n{m_confusion}')

## Algoritmo de búsqueda hacia atrás

In [8]:
#Esto parametros nos permite tener el minimo num de variables de predictoras y un mayor score
import funciones.BusquedaSecuencialAtras as bsa
bsatras = bsa.backward_sequential_search(breast_cancer, 'Diagnosis', model, 7, 6)
bsatras


### Árbol de decisión de clasificación(DecisionTreeClassifier)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(bsatras)-1, 1):
    selected_variables = bsatras.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    

    # Dividir los datos
    X = breast_cancer[selected_variables]
    objetivo= breast_cancer['Diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=4,  # Máxima profundidad del árbol
        min_samples_split=5  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=4)
    clasificador_CART.fit(X_train, objetivo)

    # Calcular la precisión del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(y_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()

Variabes predictoras: 
['mean radius', 'mean texture', 'mean smoothness', 'radius error', 'area error', 'compactness error', 'concave points error', 'fractal dimension error', 'worst texture', 'worst area', 'worst smoothness', 'worst concavity', 'worst concave points']
Precisión: 0.9982425307557118
Valores: diagnosis
1    357
0    212
Name: count, dtype: int64
Matriz de confusión: 
[[211   1]
 [  0 357]]

Variabes predictoras: 
['mean radius', 'mean texture', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst area', 'worst smoothness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']
Precisión: 1.0
Valores: diagnosis
1    357
0    212
Name: count, dtype: int64
Matriz de confusión: 
[[212   

### KNN

In [ ]:
from sklearn.naive_bayes import CategoricalNB
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 5 ejemplos más cercanos
    n_neighbors=5,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)
for k in range(0, len(bsatras)-1, 1):

    selected_variables = bsatras.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    clasificador_kNN.fit(solucion, objetivo)
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)


    # Calcular la precisión del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')

Variabes predictoras: ['mean radius', 'mean texture', 'mean smoothness', 'radius error', 'area error', 'compactness error', 'concave points error', 'fractal dimension error', 'worst texture', 'worst area', 'worst smoothness', 'worst concavity', 'worst concave points']
Precisión: 0.9507908611599297
Valores: diagnosis
1    357
0    212
Name: count, dtype: int64
Matriz de confusión: 
[[190  22]
 [  6 351]]
Variabes predictoras: ['mean radius', 'mean texture', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst area', 'worst smoothness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']
Precisión: 0.9578207381370826
Valores: diagnosis
1    357
0    212
Name: count, dtype: int64
Matriz de confusió

## Algoritmo de búsqueda hacia atrás mixta

In [ ]:
import funciones.BusquedaSecuencialAtrasMixta as bsam
busq_atras_mixta = bsam.backward_sequential_mixed_search(breast_cancer, 'Diagnosis', model, 10, 6, 10)
busq_atras_mixta

,variables,size,score
12,"[mean radius, mean texture, mean perimeter, me...",18,0.960917
20,"[mean texture, mean area, mean concavity, radi...",13,0.960797
17,"[mean texture, mean area, mean concavity, radi...",15,0.960793
34,"[concavity error, mean perimeter, symmetry err...",18,0.960738
30,"[concavity error, mean perimeter, symmetry err...",14,0.960566
16,"[mean radius, mean texture, mean area, mean co...",15,0.960555
14,"[mean radius, mean texture, mean area, mean co...",16,0.960521
33,"[concavity error, mean perimeter, symmetry err...",17,0.960286
15,"[mean radius, mean texture, mean area, mean co...",16,0.960235
24,"[mean texture, texture error, area error, wors...",13,0.960143


### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(busq_atras_mixta)-1, 1):
    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    # Realizar la búsqueda secuencial hacia atrás
    model = DecisionTreeClassifier(max_depth=8)

    # Dividir los datos
    X = breast_cancer[selected_variables]
    objetivo= breast_cancer['Diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=4,  # Máxima profundidad del árbol
        min_samples_split=5  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=4)
    clasificador_CART.fit(X_train, y_train)

    # Calcular la precisión del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Imprimir los resultados
    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()  

Variabes predictoras: 
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean concavity', 'mean concave points', 'mean symmetry', 'radius error', 'texture error', 'perimeter error', 'area error', 'compactness error', 'symmetry error', 'worst radius', 'worst compactness', 'worst concave points', 'worst symmetry', 'concavity error']
Precisión: 1.0
Valores: diagnosis
1    357
0    212
Name: count, dtype: int64
Matriz de confusión: 
[[212   0]
 [  0 357]]

Variabes predictoras: 
['mean texture', 'mean area', 'mean concavity', 'radius error', 'texture error', 'perimeter error', 'area error', 'worst radius', 'worst concave points', 'concavity error', 'mean perimeter', 'symmetry error', 'worst compactness']
Precisión: 1.0
Valores: diagnosis
1    357
0    212
Name: count, dtype: int64
Matriz de confusión: 
[[212   0]
 [  0 357]]

Variabes predictoras: 
['mean texture', 'mean area', 'mean concavity', 'radius error', 'texture error', 'perimeter error', 'area error', 'compactness er

### KNN

In [ ]:
from sklearn.naive_bayes import CategoricalNB
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 5 ejemplos más cercanos
    n_neighbors=5,
    # La cercanía viene determinada por la distancia euclídea
    metric='manhattan'
)
clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)
for k in range(0, len(busq_atras_mixta)-1, 1):

    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtiener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    clasificador_kNN.fit(solucion, objetivo)
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)


    # Calcular la precisión del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Precisión: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')

Variabes predictoras: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']
Precisión: 0.9560632688927944
Valores: diagnosis
1    357
0    212
Name: count, dtype: int64
Matriz de confusión: 
[[190  22]
 [  3 354]]
